In [ ]:
from helpers import h1
from pocket_cube.cube import Cube
from pocket_cube.constants import CORNERS
from tests import case1,case2,case3,case4
from math import log,sqrt
from random import choice
from copy import deepcopy,copy
import numpy as np
from pocket_cube.moves import Move
import numpy as np
import time

In [2]:
# Constante

N = 'N'
Q = 'Q'
PARENT = 'parent'
ACTIONS = 'actions'

In [3]:
# Funcție ce întoarce un nod nou,
def init_node(parent = None):
    return {N: 0, Q: 0, PARENT: parent, ACTIONS: {}}

In [4]:
def select_action(node, c):
    N_node = node[N]
    actions_dictionary=node[ACTIONS]
    maximum_expression=None
    action_result=None
    for action_id,child_node in actions_dictionary.items():
        quality_child_node=child_node[Q]
        simulations_child_node=child_node[N]
        expr_result=quality_child_node/simulations_child_node+c*sqrt(2*log(N_node)/simulations_child_node)
        if(maximum_expression==None):
            maximum_expression=expr_result
            action_result=action_id
        else:
            if(expr_result > maximum_expression):
                maximum_expression=expr_result
                action_result=action_id
    return action_result

In [5]:
def get_available_actions(current_state:np.ndarray,discovered_states:dict):
    available_actions=[]
    for move_number in range(6):
        new_state=Cube.move_state(current_state,move_number)
        if(tuple(new_state) not in discovered_states):
            available_actions.append(move_number)
    return available_actions

In [6]:
def mcts(initial_state:np.ndarray,budget,c,final_state:np.ndarray,heuristic):
    tree = init_node()
    reached_final_state=False
    number_of_states=1
    discovered_states={tuple(initial_state):True}
    path_length=0
    can_break_budget_loop=False
    for i in range(budget):
        node = tree
        current_state=initial_state
        number_of_moves=0
        budget_path_length=0
        """
        Etapa de tree policy:
        cat timp nu am ajuns inca la un nod frunza
        daca la starea curenta nu avem toate nodurile expandate ne oprim
        altfel, alegem cel mai bun nod
        """
        while(node[ACTIONS]!={}):
            if(get_available_actions(current_state,discovered_states)!=[]):
                break
            best_move=select_action(node,c)
            current_state=Cube.move_state(current_state,best_move)
            node=node[ACTIONS][best_move]
            budget_path_length +=1
        """
        Construim un nod pe baza unei actiuni alese aleator. 
        Trebuie sa nu fim in starea finala si sa mai avem noduri de explorat
        """
        available_actions=get_available_actions(current_state,discovered_states)
        if((current_state.tolist()!=final_state.tolist()) and available_actions):
            random_move=choice(available_actions)
            child_node=init_node(node)
            current_state=Cube.move_state(current_state,random_move)
            node[ACTIONS][random_move]=child_node
            node=child_node
            number_of_states +=1
            discovered_states[tuple(current_state)]=True
            budget_path_length +=1
        if(current_state.tolist()==final_state.tolist()):
            reached_final_state=True
            path_length=budget_path_length
        """
        Etapa default policy
        Din nodul curent se simuleaza jocul, alegandu-se mutari in mod aleator
        """
        maximum_value_reward=0
        rollout_discovered_states=copy(discovered_states)
        while((current_state.tolist()!=final_state.tolist()) and (number_of_moves<14)):
            maximum_value_reward=max(maximum_value_reward,8.5-heuristic(current_state,final_state))
            available_actions=get_available_actions(current_state,rollout_discovered_states)
            if(available_actions==[]):
                break
            random_move=choice(available_actions)
            current_state=Cube.move_state(current_state,random_move)
            number_of_moves +=1
            number_of_states+=1
            rollout_discovered_states[tuple(current_state)]=True
        maximum_value_reward=max(maximum_value_reward,8.5-heuristic(current_state,final_state))
        reward=maximum_value_reward
        while(node!=None):
            node[N] += 1
            node[Q] +=reward
            node=node[PARENT]
        if(reached_final_state==True):
            break
    if(reached_final_state):
        return number_of_states,path_length
    return None,None

In [7]:
def run_mcts(budget,c,iterations,case,heuristic):
    sum_path_length,sum_number_of_states,sum_total_time=0,0,0
    successes=0
    for i in range(iterations):
        start_time=time.time()
        number_of_states,path_length=mcts(Cube(case).state,budget,c,Cube(case).goal_state,heuristic)
        end_time=time.time()
        if(number_of_states and path_length):
            sum_total_time += (end_time-start_time)
            sum_path_length += path_length
            sum_number_of_states+=number_of_states
            successes+=1
    if(successes!=0):
        return sum_total_time/successes,sum_path_length/successes,sum_number_of_states/successes
    return None

def get_measurements(heuristic):
    measurements_array=np.zeros((4,3,2,4))
    for k,case in enumerate([case1,case2,case3,case4]):
        for i,c in enumerate([0.1,0.5]):
            for j,budget in enumerate([1000,5000,10000,20000]):
                result_mcts=run_mcts(budget,c,25,case,heuristic)
                if(result_mcts):
                    measurements_array[k][0][i][j]=result_mcts[0]
                    measurements_array[k][1][i][j]=result_mcts[1]
                    measurements_array[k][2][i][j]=result_mcts[2]
    return measurements_array

In [8]:
measurements_array_h1=get_measurements(h1)

In [25]:
measurements_array_h1[0][2]

array([[ 3042.5       , 15342.30434783, 16842.95454545, 43497.76      ],
       [ 3492.24      ,  2881.72      ,  3250.4       ,  3317.56      ]])

In [28]:
measurements_array_h1[1][2]

array([[  5381.        ,  34608.2       ,  66704.        ,
        118521.26666667],
       [  8888.5       ,  30497.17647059,  32661.69565217,
         28673.04      ]])

In [31]:
measurements_array_h1[2][2]

array([[  9883.66666667,  22179.625     ,  64380.16666667,
        121634.85714286],
       [     0.        ,  36011.        ,  42258.1875    ,
        109931.52      ]])

In [34]:
measurements_array_h1[3][2]

array([[  3980.75      ,  25452.        , 100726.11111111,
        151946.        ],
       [     0.        ,  19397.        ,      0.        ,
        128147.        ]])

In [13]:
def h2(current_state:np.ndarray,final_state:np.ndarray):
    result=0
    for _,(indices,_) in CORNERS.items():
        if(current_state[indices].tolist()!=final_state[indices].tolist()):
            result += 1
    return result
measurements_array_h2=get_measurements(h2)

In [35]:
measurements_array_h2[0][0]*1000

array([[ 379.0937337 , 2158.06903158, 4968.61105495, 8406.04752586],
       [ 699.42053996, 1113.82616085, 2292.16013908, 1347.03988075]])

In [36]:
measurements_array_h2[0][1]

array([[ 6.45454545,  9.42857143, 13.55555556, 12.23809524],
       [ 5.        ,  5.26086957,  5.4       ,  5.        ]])

In [37]:
measurements_array_h2[0][2]

array([[ 3388.72727273, 16172.        , 33145.11111111, 51683.9047619 ],
       [ 6497.10526316, 10169.        , 19879.64      , 11950.96      ]])

In [38]:
measurements_array_h2[1][0]*1000

array([[  635.00084877,  4518.88728142,  9228.63049805, 19911.78184748],
       [  834.00111198,  4049.40162897,  3760.80936477,  5401.56002045]])

In [39]:
measurements_array_h2[1][1]

array([[ 9.        , 21.5       , 22.375     , 24.375     ],
       [ 7.        ,  8.1       ,  8.33333333,  7.64      ]])

In [40]:
measurements_array_h2[1][2]

array([[  5051.6       ,  28864.        ,  55084.125     ,
        106743.3125    ],
       [  7612.4       ,  33993.65      ,  31756.33333333,
         44424.52      ]])

In [41]:
measurements_array_h2[2][0]*1000

array([[    0.        ,  4601.51425004, 15739.18520321, 26088.2647414 ],
       [    0.        ,  4887.2897625 , 11202.46208631, 15144.2911307 ]])

In [42]:
measurements_array_h2[2][1]

array([[ 0.        , 18.5       , 27.18181818, 31.94736842],
       [ 0.        ,  9.85714286, 10.69230769, 11.4       ]])

In [43]:
measurements_array_h2[2][2]

array([[     0.        ,  31652.75      ,  87532.72727273,
        134133.05263158],
       [     0.        ,  41379.85714286,  84834.        ,
        113679.4       ]])

In [44]:
measurements_array_h2[3][0]*1000

array([[    0.        ,  7056.66637421, 13538.56015205, 22300.69285173],
       [    0.        ,  2691.03264809,  1721.00257874, 25532.31186867]])

In [45]:
measurements_array_h2[3][1]

array([[ 0.        , 30.33333333, 29.90909091, 37.        ],
       [ 0.        , 11.        , 13.        , 14.6       ]])

In [46]:
measurements_array_h2[3][2]

array([[     0.        ,  39847.        ,  70301.18181818,
        105263.61538462],
       [     0.        ,  22674.        ,  14704.        ,
        166703.8       ]])